In [1]:
import tensorflow as tf
#from tensorflow import keras
#from tensorflow.keras import layers
#import tensorflow_datasets as tfds
import tensorflow.keras.optimizers
import tensorflow.keras.metrics
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cf
import pickle 
from datetime import date
import xarray as xr
import netCDF4 as nc
#from u_interpolate_small import regrid_irregular_quick
#import u_interpolate_small as uint
from ndays import numOfDays
import glob
import calendar
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import math

2025-02-13 17:04:41.074745: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739466281.084540 1341938 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739466281.087482 1341938 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-13 17:04:41.097612: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Define domain and time period
start_lat = 5  #
end_lat = 25   #
start_lon = -18 #
end_lon = -2  #
start_year = '2005'
end_year = '2019'
start_month = '08' #6
end_month = '08'  #9
start_day = '01'
end_day = '31'
lead_time = 1


# find ndays
t = 31   #122 #numOfDays(date1, date2)+1


In [3]:
# get native MSG grid (core)
coords_filename= '../nxny1640_580_nxnyds164580_blobdx0.04491576_area4_n23_20_32.nc'#[0]  # this is /prj/Africa_cloud/geoloc/*.npz on the Linux system
msg_file = xr.open_dataset(coords_filename).squeeze() # pick any convective core file from ch9_wavelet. 
mlon = msg_file['lons_mid'].values
mlat = msg_file['lats_mid'].values
# find core indices using one file
lat_ind = np.where((mlat[:,1]>=start_lat) & (mlat[:,1]<=end_lat))[0]
lon_ind = np.where((mlon[1,:]>=start_lon) & (mlon[1,:]<=end_lon))[0]
lat = mlat[lat_ind[0]:lat_ind[-1]+1,lon_ind[0]:lon_ind[-1]+1]
lon = mlon[lat_ind[0]:lat_ind[-1]+1,lon_ind[0]:lon_ind[-1]+1]


a= 8 # 85
b= -63 # 98

image_height= 512 #lat
image_width= 512 #lon
num_channels= 3 #  core at t0-, core at t0-1,  

In [4]:
# read in cores data
list_of_input_file_names = []

for m in range(int(start_month),int(end_month)+1,1): # 1 to include the end month
    if m<10:
        month = str(m).zfill(2)
    else:
        month = str(m)  
    for d in range(1,calendar.monthrange(int(start_year), m)[1]+1,1): # 1 to include the end month
        if d<10:
            day = str(d).zfill(2)
        else:
            day = str(d)  
            #core_filename= '/prj/nflics/hist_cores/2008/06/01/Hist_cores_wa_200806011045.nc'
        dir_name = '/prj/nflics/hist_cores/'+start_year+'/'+month+'/'+day+'/'
        all_file_names=sorted(glob.glob(dir_name+"Hist_cores_wa_*.nc"));
        list_of_input_file_names.append(all_file_names) # all days in month
                  
list_of_input_files=[]                
for a in range(0,len(list_of_input_file_names),1):
    list_of_input_files = list_of_input_files+list_of_input_file_names[a]

list_of_output_files=list_of_input_files[4*lead_time:]
list_of_input_files=list_of_input_files[0:-4*lead_time]
    
print(len(list_of_input_files))

2948


In [5]:
def load_sequence(i):
     
    # Load 3x4 consecutive input frames
    tir = np.zeros((3*4,len(lat[:,1]),len(lon[1,:])),dtype=float)

    for offset in range(3*4):
        with nc.Dataset(list_of_input_files[i + offset]) as ds:
            tir_temp =  ds.variables['msg_Tir'][:]
            tir[i + offset,:,:] = tir_temp[lat_ind[0]:lat_ind[-1]+1,lon_ind[0]:lon_ind[-1]+1]/10000    

    tir_t_0 = tir[:, a:,:b]
    ind_tir = np.where(tir_t_0>-0.01)
    tir_t_0[ind_tir] = 0
    tir_t_0[np.isnan(tir_t_0)] = 0
    tir_t_0 = np.round(tir_t_0/-173,4)
    tir_t_0_minus_2 = tir_t_0[0:-4*2]
    tir_t_0_minus_1 = tir_t_0[4*1:-4*1]
    tir_t_0 = tir_t_0[4*2:]
    # define training array
    total_samples= int(tir_t_0.shape[0])
    x_train= np.zeros((total_samples,image_height,image_width, num_channels))
    x_train[:,:,:,0]= tir_t_0_minus_2
    x_train[:,:,:,1]= tir_t_0_minus_1
    x_train[:,:,:,2]= tir_t_0
    
    
    # Load the output frame
    #target data
    cores = np.zeros((i*4,len(lat[:,1]),len(lon[1,:])),dtype=float)
    time_core = np.zeros((i*4)) 

    for offset in range(4):
        core_filename = list_of_output_files[i + offset]  #'/prj/Africa_cloud/ch9_wavelet/'+str(i)+'/'+month+'/'  list_of_files[l]
        print(core_filename)
        ds = xr.open_dataset(core_filename).squeeze() 
        core_temp = ds['msg_cores'].values
        cores[i + offset,:,:] = core_temp[lat_ind[0]:lat_ind[-1]+1,lon_ind[0]:lon_ind[-1]+1]

    ind = np.where(cores>0)
    cores[ind] = 1
    cores_t_0 = cores[:, a:,:b]
    cores_t_plus_xhr = cores[:,a:,:b]
    prediction_time = time_core
    
    y_train = cores_t_plus_xhr

    time_of_day_tr= np.zeros((len(prediction_time),image_height, image_width,1))

    for t in range(0,len(prediction_time)):
        time_of_day = float(str(prediction_time[t])[-6:])/2345
        time_of_day_tr[t,:,:,:]=round(np.sin(time_of_day*math.pi),2)
        print(time_of_day_tr[t,0,0,0])

    # define input and output tensors
    input_tensor = [x_train,time_of_day_tr]
    output_tensor=tf.expand_dims(y_train, axis=3)

    return input_tensor, output_tensor



In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D)
from tensorflow.keras import layers


#meta_input_shape= 32x32x8

# define conv layer
def conv_layer(x_in, filters, gn_num, strides = 1, shape = (image_height,image_width), name=None):
    x = Conv2D(filters, 5, strides=strides, padding='same')(x_in)
    x = Activation('relu', name = name)(x)
   ## x = layers.BatchNormalization()(x)
    #x = tfa.layers.GroupNormalization(groups=gn_num, axis=3)(x) # could replace this with BatchNorm
    return x

# def basic unet structure
def unet_basic(input_shape=(image_height, image_width, num_channels), meta_input_shape=(image_height, image_width,1) ,chan_num=num_channels):

    inputs = Input(shape=input_shape)    # 256 x 256 x 2
    meta_inputs = Input(shape=meta_input_shape)

    # downsample
    down2 = conv_layer(inputs, 4, 1)     # 256 x 256 x 4
    down2_pool = MaxPooling2D((2, 2), strides=None)(down2)   # 128 x 128 x 4

    down3 = conv_layer(down2_pool, 8, None) # 128 x 128 x 8
    down3_pool = MaxPooling2D((4, 4), strides=None)(down3) # 32 x 32 x 8
    #print(down3_pool.shape)
 
    center = conv_layer(down3_pool, 16, None) # 32 x 32 x 16
    center = conv_layer(center, 8, None) # 32 x 32 x 8
  
    up3 = UpSampling2D((4, 4))(center) # 128 x 128 x 8
    up3 = concatenate([down3, up3], axis=3) # 128 x 128 x 16
    up3 = conv_layer(up3, 4, None) # 128 x 128 x 4

    up2 = UpSampling2D((2, 2))(up3) # 256 x 256 x 4
    up2 = concatenate([down2, up2, meta_inputs], axis=3) # 256 x 256 x 9
#    up2 = concatenate([down2, up2], axis=3) # 256 x 256 x 8
    up2 = conv_layer(up2, 4, None) # 256 x 256 x 4
  
    # predict
    output = Conv2D(1, (1, 1))(up2)
#    time_conv = layers.concatenate([output, meta_inputs]) # 256x256x2
    #time_conv = layers.concatenate([up2, meta_inputs]) # 256x256x2
  #  output1 = conv_layer(time_conv, 1, None) # 256 x 256 x 1
#    output2 = layers.Activation('sigmoid', dtype='float32', name='prob_pred')(output1)
    output1 = layers.Activation('sigmoid', dtype='float32', name='prob_pred')(output)
    
    # create model object
    unet = Model(inputs=[inputs,meta_inputs], outputs=output1, name = 'prob_map_unet')
    return unet


unet_model = unet_basic(input_shape=(image_height, image_width, num_channels),meta_input_shape=(image_height, image_width,1), chan_num=num_channels)


I0000 00:00:1739466423.623796 1341938 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1027 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:02:00.0, compute capability: 8.6


In [7]:
def _create_mean_filter(half_num_rows, half_num_columns, num_channels):
    """Creates convolutional filter that computes mean.

    M = number of rows in filter
    N = number of columns in filter
    C = number of channels

    :param half_num_rows: Number of rows on either side of center.  This is
        (M - 1) / 2.
    :param half_num_columns: Number of columns on either side of center.  This
        is (N - 1) / 2.
    :param num_channels: Number of channels.
    :return: weight_matrix: M-by-N-by-C-by-C numpy array of filter weights.
    """

    num_rows = 2 * half_num_rows + 1
    num_columns = 2 * half_num_columns + 1
    weight = 1. / (num_rows * num_columns)

    return np.full(
        (num_rows, num_columns, num_channels, num_channels), weight,
        dtype=np.float32
    )

def FSS_loss(target_tensor, prediction_tensor):
    
    half_window_size_px=2
    use_as_loss_function=True 
    #mask_matrix
    function_name=None
    test_mode=False
    """Fractions skill score (FSS).

    M = number of rows in grid
    N = number of columns in grid

    :param half_window_size_px: Number of pixels (grid cells) in half of
        smoothing window (on either side of center).  If this argument is K, the
        window size will be (1 + 2 * K) by (1 + 2 * K).
    :param use_as_loss_function: Boolean flag.  FSS is positively oriented
        (higher is better), but if using it as loss function, we want it to be
        negatively oriented.  Thus, if `use_as_loss_function == True`, will
        return 1 - FSS.  If `use_as_loss_function == False`, will return just
        FSS.
    :param mask_matrix: M-by-N numpy array of Boolean flags.  Grid cells marked
        "False" are masked out and not used to compute the loss.
    :param function_name: Function name (string).
    :param test_mode: Leave this alone.
    :return: loss: Loss function (defined below).
    """

    weight_matrix = _create_mean_filter(
        half_num_rows=half_window_size_px,
        half_num_columns=half_window_size_px, num_channels=1
    )
       
    #print(target_tensor.shape)
    #print(prediction_tensor.shape)
    #print(weight_matrix.shape)
        
    """Computes loss (fractions skill score).

        :param target_tensor: Tensor of target (actual) values.
        :param prediction_tensor: Tensor of predicted values.
        :return: loss: Fractions skill score.
    """

    smoothed_target_tensor = K.conv2d(
        x=target_tensor, kernel=weight_matrix,
        padding='same', strides=(1, 1), data_format='channels_last'
    )

    smoothed_prediction_tensor = K.conv2d(
        x=prediction_tensor, kernel=weight_matrix,
        padding='same', strides=(1, 1), data_format='channels_last'
    )

    #smoothed_target_tensor = smoothed_target_tensor * eroded_mask_matrix
    #smoothed_prediction_tensor = smoothed_prediction_tensor * eroded_mask_matrix)

    actual_mse = K.mean(
        (smoothed_target_tensor - smoothed_prediction_tensor) ** 2
    )
    reference_mse = K.mean(
        smoothed_target_tensor ** 2 + smoothed_prediction_tensor ** 2
    )

    if use_as_loss_function:
        return actual_mse / reference_mse

    return 1. - actual_mse / reference_mse

    if function_name is not None:
        loss.__name__ = function_name

In [8]:
from tensorflow.python.keras.engine import data_adapter

def _is_distributed_dataset(ds):
    return isinstance(ds, data_adapter.input_lib.DistributedDatasetSpec)

data_adapter._is_distributed_dataset = _is_distributed_dataset

tf.config.run_functions_eagerly(True)

In [9]:
# 2. Build dataset
dataset = tf.data.Dataset.range(len(list_of_input_files) - 3*4) 
dataset = dataset.map(load_sequence, num_parallel_calls=tf.data.AUTOTUNE)
#dataset = dataset.shuffle(buffer_size=100)  # shuffle if desired
dataset = dataset.batch(8)                 # batch size
dataset = dataset.prefetch(tf.data.AUTOTUNE)



/home/jawahm/miniforge3/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


TypeError: in user code:

    File "/tmp/ipykernel_1341938/2162881432.py", line 7, in load_sequence  *
        with nc.Dataset(list_of_input_files[i + offset]) as ds:

    TypeError: list indices must be integers or slices, not SymbolicTensor


In [ ]:
unet_model.compile(optimizer=tensorflow.keras.optimizers.Adam(),
                  loss=FSS_loss,
                  metrics=[tf.keras.metrics.Accuracy()])
#                  metrics=metric_list)

In [ ]:
epochs =  20

# Fit the model to the training data.
model_history = unet_model.fit(dataset,epochs=epochs)
#    callbacks=[early_stopping, reduce_lr],